# Module 1 + Module 2 (no tuple)
The problem is formally defined as follows:
- a depot 0 where all vehicles are located in the beginning
- a set of customers $C = {1,...,N}$, we define locations $L = {0} + C$
- a demand $d_i$ for each customer $i \in C$ (the depot has no demand)
- a time window $[a_i,b_i]$ with $0<a_i<b_i$ for each location $i \in L$ (including the depot)
- a service time $s_i$ for each customer $i \in C$ (the depot has zero service time)
- costs $c_{ij}$ and time $t_{ij}$ for travelling from location $i$ to location $j$
- a homogeneous fleet $K = {1,...,|K|}$ of vehicles with load capacity $Q$

We want to find
- a route for each vehicle, i.e., an ordered sequence of customers
- with minimal total routing costs/time, such that
- each customer is visited exactly once
- the start times of the services have to be within the depot and customer time windows (potentially introducing waiting times between services),
- and the vehicle capacity is never exceeded

## Instance Configuration

In [438]:
import math
import random
import numpy as np

### INSTANCE CONFIGURATION

numCustomers = 5
maxNumVehicles = 2

vehicleCapacity = 30
demandRange = (2, 5)

timeHorizon = 100 # tmax
timeWindowWidthRange = (10, 15)
serviceTimeRange = (2, 4)

## Instance Generation

In [439]:
random.seed(0)

depot = 0
customers = [*range(1, numCustomers + 1)]
locations = [depot] + customers
connections = [(i, j) for i in locations for j in locations if i != j]
vehicles = [*range(1, maxNumVehicles + 1)]

# create random depot and customer locations in the Euclidian plane (1000x1000)
points = [(random.randint(0, 999), random.randint(0, 999)) for i in locations]
# dictionary of Euclidean distance for each connection (interpreted as travel costs)
costs = {
    (i, j): math.ceil(
        math.sqrt(sum((points[i][k] - points[j][k]) ** 2 for k in range(2)))
    )
    for (i, j) in connections
}
maximalCosts = math.ceil(999 * math.sqrt(2))
# dictionary of travel times for each connection (related to the costs, scaled to time horizon)
travelTimes = {
    (i, j): math.ceil((costs[i, j] / maximalCosts) * timeHorizon * 0.2)
    for (i, j) in connections
}

# create random demands, service times, and time window widths in the given range
demands = {i: random.randint(demandRange[0], demandRange[1]) for i in customers}
demands[0] = 0  # depot has no demand
serviceTimes = {
    i: random.randint(serviceTimeRange[0], serviceTimeRange[1]) for i in customers
}
serviceTimes[0] = 0  # depot has no service time
timeWindowWidths = {
    i: random.randint(timeWindowWidthRange[0], timeWindowWidthRange[1])
    for i in customers
}
# vehicles are allowed to leave the depot any time within the time horizon
timeWindowWidths[0] = timeHorizon

# create time windows randomly based on the previously generated information
# such that the service at a customer can be finished within the time horizon
timeWindows = {}
timeWindows[0] = (0, 0 + timeWindowWidths[0])
for i in customers:
    start = random.randint(0, timeHorizon - serviceTimes[i] - timeWindowWidths[i] - travelTimes[i,0])
    timeWindows[i] = (start, start + timeWindowWidths[i])

## Output Visualisation

In [440]:
print("Customer Demands:")
for customer, demand in demands.items():
    print(f"Customer {customer}: Demand = {demand}")

print("\nCustomer Service Times:")
for customer, service_time in serviceTimes.items():
    print(f"Customer {customer}: Service Time = {service_time}")

print("\nCustomer Time Windows:")
for customer, window in timeWindows.items():
    print(f"Customer {customer}: Time Window = {window}")

print("\nTravel Times:")
for (i, j), time in list(travelTimes.items()):
    print(f"Travel Time from {i} to {j}: {time}")


Customer Demands:
Customer 1: Demand = 4
Customer 2: Demand = 5
Customer 3: Demand = 4
Customer 4: Demand = 3
Customer 5: Demand = 3
Customer 0: Demand = 0

Customer Service Times:
Customer 1: Service Time = 3
Customer 2: Service Time = 2
Customer 3: Service Time = 2
Customer 4: Service Time = 4
Customer 5: Service Time = 3
Customer 0: Service Time = 0

Customer Time Windows:
Customer 0: Time Window = (0, 100)
Customer 1: Time Window = (12, 26)
Customer 2: Time Window = (9, 24)
Customer 3: Time Window = (42, 56)
Customer 4: Time Window = (60, 71)
Customer 5: Time Window = (71, 83)

Travel Times:
Travel Time from 0 to 1: 8
Travel Time from 0 to 2: 8
Travel Time from 0 to 3: 12
Travel Time from 0 to 4: 6
Travel Time from 0 to 5: 11
Travel Time from 1 to 0: 8
Travel Time from 1 to 2: 14
Travel Time from 1 to 3: 8
Travel Time from 1 to 4: 7
Travel Time from 1 to 5: 6
Travel Time from 2 to 0: 8
Travel Time from 2 to 1: 14
Travel Time from 2 to 3: 14
Travel Time from 2 to 4: 7
Travel Time fr

## Model Generation
### Basic Route Problem (without Load and Time constraint)
Decision variables:
- $x_{ij} = 
\begin{cases} 
1 & \text{if a vehicle traverses arc (i, j)} \\
0 & \text{otherwise}
\end{cases}
$

The objective is to minimize the total travel time. Travel time doesn't consider the service time of each customer.

Constraints:
1. Each customer is visited exactly once, i.e., there is exactly one incoming connection and exactly one    nextCustomer connection.
2. At most $|K|$ vehicles can be used.
3. We can also compute a lower bound on the number of needed vehicles, that is the sum of all demands divided by the vehicle capacity.

$$
\text{min} \sum_{i,j \in \mathcal{L}} c_{ij}x_{ij}
$$

$$
\sum_{j \in \mathcal{C}} x_{ij} = 1 \quad \forall j \in \mathcal{C}
$$

$$
\sum_{j \in \mathcal{L}} x_{ij} = 1 \quad \forall i \in \mathcal{C}
$$

$$
\sum_{j \in \mathcal{C}} x_{0j} \leq |K|
$$

$$
\sum_{j \in \mathcal{C}} x_{0j} \geq \left\lceil \frac{\sum_{i \in \mathcal{C}} d_i}{Q} \right\rceil
$$


In [441]:
import gurobipy as gp
from gurobipy import GRB

# Function for calculate the number of vehicles needed for customers
def numVehiclesNeededForCustomers(customers):
    sumDemand = 0
    for i in customers:
        sumDemand += demands[i]
    return math.ceil(sumDemand / vehicleCapacity)

model = gp.Model("VRPTW")

# binary variables x(i,j): is 1 if some vehicle is going from node i to node j, 0 otherwise
x = model.addVars(connections, vtype=GRB.BINARY, name="x")

# objective function: minimize sum of connection costs
model.setObjective(gp.quicksum(travelTimes[i,j]*x[i,j] for i,j in connections), GRB.MINIMIZE)

# all customers have exactly one incoming and one outgoing connection
model.addConstrs((x.sum("*", j) == 1 for j in customers), name="incoming")
model.addConstrs((x.sum(i, "*") == 1 for i in customers), name="outgoing")

# vehicle limits
model.addConstr(x.sum(0, "*") <= maxNumVehicles, name="maxNumVehicles")
model.addConstr(
    x.sum(0, "*") >= numVehiclesNeededForCustomers(customers),
    name="minNumVehicles",
)


<gurobi.Constr *Awaiting Model Update*>

Then we can add Load and Time constraints in two different ways:

## Big - M Model

### Load

Decision Variable: $y_i$ for each location $i \in L$ to denote the vehicle load after picking up the demand at $i$.

Constraints:
- $y_0 = 0$
- $y_i + d_j \le y_j + Q(1 - x_{ij}) \quad \forall i \in L, j \in C, i \neq j$
- $y_i + d_j \geq y_j - M_{ij}(1 - x_{ij}) \quad \forall i \in L, j \in C, i \neq j$

### Time

Decision Variable: $z_i$ for each location $i \in L$ to denote the start time of the service at $i$ that has to be within its time window.

Constraints:
- $z_0 = 0$
- $z_i + s_i + t_{ij} \leq z_j + T(1 - x_{ij}) \quad \forall i \in L, j \in C, i \neq j$

In both case, subtours are eliminated since time/load has to monotonically increase along a route.

## Flow Model
Now, we extend the meaning of the variables used in the Big-M model. We do not associate resource states to customers anymore but instead to connections between customers.

### Load 

Decision variable: $y_{i,j}$ for each connection $(i, j)$ to denote the vehicle load after picking up the demand at $i$ and proceeding to location $j$.

Constraints:
 $$y_{0,j} = 0 \quad \forall j \in C$$
 $$\sum_{i \in L} y_{ij} + d_j = \sum_{i \in L} y_{ji} \quad \forall j \in C$$
 $$y_{ij} \geq d_ix_{ij} \quad \forall i \in C, j \in L, i \neq j$$
 $$y_{ij} \leq (Q - d_j)x_{ij} \quad \forall i \in C, j \in L, i \neq j$$

### Time
Decision variable: $z_{ij}$ for each connection $(i, j)$ to denote the start time of the service at $i$ (which must be within its time window) when immediately proceeding to location $j$.

Constraints:
 $$z_{0j} = 0 \quad \forall j \in C$$
 $$\sum_{i \in L}[z_{ij} + (s_i + t_{ij})x_{ij}] \leq \sum_{i \in L} z_{ji} \quad \forall j \in C$$
 $$z_{ij}\geq a_ix_{ij} \quad \forall i,j \in L, i \neq j$$
 $$z_{ij}\leq b_ix_{ij} \quad \forall i,j \in L, i \neq j$$

In [442]:
def addLoadConstraintsByBigM():

    y = model.addVars(locations, lb=0, ub=vehicleCapacity, name="y")
    y[0].UB = 0  # empty load at depot

    model.addConstrs(
        (
            y[i] + demands[j] <= y[j] + vehicleCapacity * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="loadBigM1",
    )
    model.addConstrs(
        (
            y[i] + demands[j]
            >= y[j] - (vehicleCapacity - demands[i] - demands[j]) * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="loadBigM2",
    )


def addLoadConstraintsByFlows():

    y = model.addVars(connections, lb=0, ub=vehicleCapacity, name="y")

    for i in customers:
        y[0, i].UB = 0

    model.addConstrs(
        (y.sum("*", j) + demands[j] == y.sum(j, "*") for j in customers),
        name="flowConservation",
    )
    model.addConstrs(
        (
            y[i, j] >= demands[i] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="loadLowerBound",
    )
    model.addConstrs(
        (
            y[i, j] <= (vehicleCapacity - demands[j]) * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="loadUpperBound",
    )


def addTimeConstraintsByBigM():

    z = model.addVars(locations, name="z")
    for i in locations:
        z[i].LB = timeWindows[i][0]
        z[i].UB = timeWindows[i][1]

    model.addConstrs(
        (
            z[i] + serviceTimes[i] + travelTimes[i, j]
            <= z[j]
            + (
                timeWindows[i][1]
                + serviceTimes[i]
                + travelTimes[i, j]
                - timeWindows[j][0]
            )
            * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="timeBigM",
    )


def addTimeConstraintsByFlows():

    z = model.addVars(connections, lb=0, name="z")

    for (i, j) in connections:
        z[i, j].UB = timeWindows[i][1]

    model.addConstrs(
        (
            gp.quicksum(
                z[i, j] + (serviceTimes[i] + travelTimes[i, j]) * x[i, j]
                for i in locations
                if (i, j) in connections
            )
            <= z.sum(j, "*")
            for j in customers
        ),
        name="flowConservation",
    )
    model.addConstrs(
        (
            z[i, j] >= timeWindows[i][0] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="timeWindowStart",
    )
    model.addConstrs(
        (
            z[i, j] <= timeWindows[i][1] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="timeWindowEnd",
    )


In [443]:
### MODEL CONFIGURATION
loadModelType = 1  # 1: big-M, 2: flow
timeModelType = 1  # 1: big-M, 2: flow

if loadModelType == 1:
    addLoadConstraintsByBigM()
elif loadModelType == 2:
    addLoadConstraintsByFlows()

if timeModelType == 1:
    addTimeConstraintsByBigM()
elif timeModelType == 2:
    addTimeConstraintsByFlows()

#model.params.Threads = 4
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i3-1000NG4 CPU @ 1.10GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 87 rows, 42 columns and 285 nonzeros
Model fingerprint: 0x6298a233
Variable types: 12 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [3e+00, 1e+01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 9e+01]
Found heuristic solution: objective 57.0000000
Presolve removed 64 rows and 28 columns
Presolve time: 0.00s
Presolved: 23 rows, 14 columns, 87 nonzeros
Variable types: 3 continuous, 11 integer (11 binary)

Root relaxation: objective 5.100000e+01, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      51.

## Output Generation

In [444]:
if model.SolCount >= 1:

    print("Objective Value: ", model.objVal)

    usedConnections = [(i, j) for (i, j) in x.keys() if x[i, j].X > 0.5]

    # create a dict for the next customer based on the current one
    # (note that the depot in general has multiple outgoing connections)
    nextCustomer = {}
    for (i, j) in usedConnections:
        if i == 0:
            if 0 not in nextCustomer.keys():
                nextCustomer[0] = []
            nextCustomer[0].append(j)
        else:
            nextCustomer[i] = j

    print(f"Solution contains {len(nextCustomer[0])} routes:")
    routeNumber = 0
    visitedCustomers = [False] * (numCustomers + 1)
    for firstCustomer in nextCustomer[0]:
        print(f"Route {routeNumber}: 0 -> ", end="")
        vehicleLoad = 0
        time = travelTimes[0, firstCustomer]
        violatedTimeWindows = False
        currentCustomer = firstCustomer
        while currentCustomer != 0:
            print(f"{currentCustomer} (L:{vehicleLoad}, T:{time}) -> ", end="")
            visitedCustomers[currentCustomer] = True
            vehicleLoad += demands[currentCustomer]
            time = max(time, timeWindows[currentCustomer][0])
            if time > timeWindows[currentCustomer][1]:
                violatedTimeWindows = True
            time += (
                serviceTimes[currentCustomer]
                + travelTimes[currentCustomer, nextCustomer[currentCustomer]]
            )
            currentCustomer = nextCustomer[currentCustomer]
        print(f"0 (L:{vehicleLoad}/{vehicleCapacity}, T:{time})")
        if vehicleLoad > vehicleCapacity:
            print("Vehicle capacity is exceeded!")
        if violatedTimeWindows:
            print("Time windows are violated!")
        routeNumber += 1

    print("Unvisited customers: ", end="")
    for c in customers:
        if visitedCustomers[c] == False:
            print(f"{c}, ", end="")

Objective Value:  51.0
Solution contains 2 routes:
Route 0: 0 -> 1 (L:0, T:8) -> 3 (L:4, T:23) -> 5 (L:8, T:47) -> 0 (L:11/30, T:85)
Route 1: 0 -> 2 (L:0, T:8) -> 4 (L:5, T:18) -> 0 (L:8/30, T:70)
Unvisited customers: 

T[i] tempo di arrivo al nodo i

z[i] tempo in cui puo iniziare il servizio in i

# Module 1 + Module 2 

## Input Generation

In [445]:
import gurobipy as gp
from gurobipy import GRB
import math
import random

### INSTANCE CONFIGURATION
numCustomers = 5
maxNumVehicles = 2

vehicleCapacity = 30
demandRange = (2, 5)

timeHorizon = 100
timeWindowWidthRange = (10, 15)
serviceTimeRange = (2, 4)
random.seed(0)

depot = 0
customers = [*range(1, numCustomers + 1)]
locations = [depot] + customers
connections = [(i, j) for i in locations for j in locations if i != j]
vehicles = [*range(1, maxNumVehicles + 1)]

# create random depot and customer locations in the Euclidian plane (1000x1000)
points = [(random.randint(0, 999), random.randint(0, 999)) for i in locations]
# dictionary of Euclidean distance for each connection (interpreted as travel costs)
costs = {
    (i, j): math.ceil(
        math.sqrt(sum((points[i][k] - points[j][k]) ** 2 for k in range(2)))
    )
    for (i, j) in connections
}
maximalCosts = math.ceil(999 * math.sqrt(2))
# dictionary of travel times for each connection (related to the costs, scaled to time horizon)
travelTimes = {
    (i, j): math.ceil((costs[i, j] / maximalCosts) * timeHorizon * 0.2)
    for (i, j) in connections
}

# create random demands, service times, and time window widths in the given range
demands = {i: random.randint(demandRange[0], demandRange[1]) for i in customers}
demands[0] = 0  # depot has no demand
serviceTimes = {
    i: random.randint(serviceTimeRange[0], serviceTimeRange[1]) for i in customers
}
serviceTimes[0] = 0  # depot has no service time
timeWindowWidths = {
    i: random.randint(timeWindowWidthRange[0], timeWindowWidthRange[1])
    for i in customers
}
# vehicles are allowed to leave the depot any time within the time horizon
timeWindowWidths[0] = timeHorizon

# create time windows randomly based on the previously generated information
# such that the service at a customer can be finished within the time horizon
timeWindows = {}
timeWindows[0] = (0, 0 + timeWindowWidths[0])
for i in customers:
    start = random.randint(0, timeHorizon - serviceTimes[i] - timeWindowWidths[i] - travelTimes[i,0])
    timeWindows[i] = (start, start + timeWindowWidths[i])

def numVehiclesNeededForCustomers(customers):
    sumDemand = 0
    for i in customers:
        sumDemand += demands[i]
    return math.ceil(sumDemand / vehicleCapacity)

# Define the restricted tuples R
R = [
    (1,3,5)
]

## Model Generation

In [446]:
model = gp.Model("VRPTW")

# Decision Variables
x = model.addVars(connections, vtype=GRB.BINARY, name="x")
y = model.addVars(locations, lb=0, ub=vehicleCapacity, name="y")
y[0].UB = 0
z = model.addVars(locations, name="z")

# Objective Function
model.setObjective(gp.quicksum(travelTimes[i,j]*x[i,j] for i,j in connections), GRB.MINIMIZE)

# all customers have exactly one incoming and one outgoing connection
model.addConstrs((x.sum("*", j) == 1 for j in customers), name="incoming")
model.addConstrs((x.sum(i, "*") == 1 for i in customers), name="outgoing")

# vehicle limits
model.addConstr(x.sum(0, "*") <= maxNumVehicles, name="maxNumVehicles")
model.addConstr(
    x.sum(0, "*") >= numVehiclesNeededForCustomers(customers),
    name="minNumVehicles",
)

model.addConstrs(
    (
        y[i] + demands[j] <= y[j] + vehicleCapacity * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="loadBigM1",
)
model.addConstrs(
    (
        y[i] + demands[j]
        >= y[j] - (vehicleCapacity - demands[i] - demands[j]) * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="loadBigM2",
)

for i in locations:
    z[i].LB = timeWindows[i][0]
    z[i].UB = timeWindows[i][1]

model.addConstrs(
    (
        z[i] + serviceTimes[i] + travelTimes[i, j]
        <= z[j]
        + (
            timeWindows[i][1]
            + serviceTimes[i]
            + travelTimes[i, j]
            - timeWindows[j][0]
        )
        * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="timeBigM",
)

# Add the new constraint for the set R
for (i, j, l) in R:
    model.addConstr(
        gp.quicksum(x[i, k] for k in customers if k != i and k != j) + 
        gp.quicksum(x[k, j] for k in customers if k != i and k != j) - 
        1 >= x[i, j] + x[j, l] - 1,
        name=f"no_shared_vehicle_{i}_{j}_{l}"
    )

model.params.Threads = 4
model.optimize()

Set parameter Threads to value 4
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i3-1000NG4 CPU @ 1.10GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 88 rows, 42 columns and 293 nonzeros
Model fingerprint: 0xe68445c6
Variable types: 12 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [3e+00, 1e+01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 9e+01]
Found heuristic solution: objective 61.0000000
Presolve removed 80 rows and 38 columns
Presolve time: 0.00s
Presolved: 8 rows, 4 columns, 24 nonzeros
Variable types: 0 continuous, 4 integer (4 binary)
Found heuristic solution: objective 57.0000000

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 2: 57 61 

Optimal solution found (tolerance 1.00e-04)
Best ob

## Output Generation

In [447]:
if model.SolCount >= 1:

    print("Objective Value: ", model.objVal)

    usedConnections = [(i, j) for (i, j) in x.keys() if x[i, j].X > 0.5]

    # create a dict for the next customer based on the current one
    # (note that the depot in general has multiple outgoing connections)
    nextCustomer = {}
    for (i, j) in usedConnections:
        if i == 0:
            if 0 not in nextCustomer.keys():
                nextCustomer[0] = []
            nextCustomer[0].append(j)
        else:
            nextCustomer[i] = j

    print(f"Solution contains {len(nextCustomer[0])} routes:")
    routeNumber = 0
    visitedCustomers = [False] * (numCustomers + 1)
    for firstCustomer in nextCustomer[0]:
        print(f"Route {routeNumber}: 0 -> ", end="")
        vehicleLoad = 0
        time = travelTimes[0, firstCustomer]
        violatedTimeWindows = False
        currentCustomer = firstCustomer
        while currentCustomer != 0:
            print(f"{currentCustomer} (L:{vehicleLoad}, T:{time}) -> ", end="")
            visitedCustomers[currentCustomer] = True
            vehicleLoad += demands[currentCustomer]
            time = max(time, timeWindows[currentCustomer][0])
            if time > timeWindows[currentCustomer][1]:
                violatedTimeWindows = True
            time += (
                serviceTimes[currentCustomer]
                + travelTimes[currentCustomer, nextCustomer[currentCustomer]]
            )
            currentCustomer = nextCustomer[currentCustomer]
        print(f"0 (L:{vehicleLoad}/{vehicleCapacity}, T:{time})")
        if vehicleLoad > vehicleCapacity:
            print("Vehicle capacity is exceeded!")
        if violatedTimeWindows:
            print("Time windows are violated!")
        routeNumber += 1

    print("Unvisited customers: ", end="")
    for c in customers:
        if visitedCustomers[c] == False:
            print(f"{c}, ", end="")

Objective Value:  57.0
Solution contains 2 routes:
Route 0: 0 -> 1 (L:0, T:8) -> 4 (L:4, T:22) -> 0 (L:7/30, T:70)
Route 1: 0 -> 2 (L:0, T:8) -> 3 (L:5, T:25) -> 5 (L:9, T:47) -> 0 (L:12/30, T:85)
Unvisited customers: 

# Module 3
The Module 3 is related to the *Last Mile Delivery Problem with Parcel Lockers*. This problem is a variant of VRP where vehicles have to deliver parcels to a set of consumers that can be served directly at their home or, as an alternative, to a locker station.

So, the objective of the problem is to decide which locker stations to open to minimize the delivery time.

For this reason, we have to update the previous notation. Now we have two sets of nodes:
- $N_C$ is the set of customers
- $N_L$ is the set of lockers

So the complete direct graph will be $G = (N, A)$ where $N = {0} \cup N_C \cup N_L$ and $A$ is the arc set.

In addition, the problem requires that a customer is willing to travel from home to collecthis package iff the distance between the customer and the locker is lower or equal to $d_{max}$.

To separately account for arcs travelled by consumers, we define as $N_L^c \in N_L$ the subset of potential parcel lockers located at a distance lower than or equal to $d_{max}$ from customer $c$.

We can assume that locker stations have unbounded capacity.

As in the previous module we can define the following **decision variables**:
- $x_{ij} = 
\begin{cases} 
1 & \text{if a vehicle traverses arc (i, j)} \\
0 & \text{otherwise}
\end{cases}
$
- a continuous variable $z_{ij}$ indicating the time of arrival of a vehicle at node $j$ when arriving from $i$
- for each set of nodes $S \in N$, where $N$ is the total set of nodes, let $$\delta^+(S) = \{(i, j) \in A : i \in S, j \notin S\}$$ $$\delta^-(S) = \{(i, j) \in A : i \notin S, j \in S\}$$
be the set of arcs leaving and entering set S. So, if $S = {i}$ then $\delta^+(S) = \delta^+({i})$
- The decision to open a locker at any location $l \in N_L$ is regulated by the binary variable 
$$y_{l} = 
\begin{cases} 
1 & \text{if the locker is opened} \\
0 & \text{otherwise}
\end{cases}
$$
- $w_{cl} = 
\begin{cases} 
1 & \text{if customer c travels to locker l} \\
0 & \text{otherwise (the consumer receives the parcel directly at
home)}
\end{cases}
$



In [448]:
# Create the model
model = gp.Model("VRPPL")

# Set of client and locker nodes
N_C = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # clients
N_L = [11, 12, 13]  # lockers
N = [0] + N_C + N_L  # N = {0} ∪ N_C ∪ N_L (0 is the depot)
num_nodes = len(N)
A = [(i, j) for i in N for j in N if i != j]
vehicles = [1, 2, 3]
K = len(vehicles)
t = {0: 0, 1: 2, 2: 3, 3: 2, 4: 1, 5: 2, 6: 4, 7: 2, 8: 3, 9: 2, 10: 1, 11: 3, 12: 5, 13: 2}
vehicle_capacity = 10
demand = {1: 3, 2: 4, 3: 2, 4: 1, 5: 2, 6: 4, 7: 2, 8: 3, 9: 2, 10: 1}

# maximum distance
d_max = 10

# Distances between nodes
np.random.seed(0)  # Per la riproducibilità
distance_matrix = {}

for i in range(num_nodes):
    for j in range(i + 1, num_nodes):
        distance = np.random.randint(1, 31)  # Distanza casuale tra 1 e 30
        distance_matrix[(N[i], N[j])] = distance
        distance_matrix[(N[j], N[i])] = distance  # Distanza bidirezionale

# Function to return the distance between two nodes
def distance(i, j):
    if i == j:
        return 0
    return distance_matrix.get((i, j), distance_matrix.get((j, i), float('inf')))

time_windows = {
    1: (0, 40),
    2: (0, 40),
    3: (0, 40),
    4: (0, 40),
    5: (0, 40),
    6: (0, 40),
    7: (0, 40),
    8: (0, 40),
    9: (0, 40),
    10: (0, 40)
}


# Decision Variables

# 1 if arc (i, j) is travelled
x = model.addVars(N, N, vtype=GRB.BINARY, name="x")

# Arriving time at node j
z = model.addVars(N, N, vtype=GRB.CONTINUOUS, name="z")

# 1 if locker i is opened
y = model.addVars(N_L, vtype=GRB.BINARY, name="y")

# 1 if client c goes to locker i
w = model.addVars(N_C, N_L, vtype=GRB.BINARY, name="w")

# Set of accessible lockers for each customer, respecting d_max
N_L_c = {c: [l for l in N_L if distance(c, l) <= d_max] for c in N_C}

# vehicle load after picking up the demand at i
p = model.addVars(N, lb=0, ub=vehicle_capacity, name="p")
p[0].UB = 0

# Objective function
model.setObjective(
    gp.quicksum(distance(i, j) * x[i, j] for i in N for j in N if i != j),
    GRB.MINIMIZE
)

So, we can plot, for each customer, the set of lockers that can be reached

In [449]:
N_L_c

{1: [],
 2: [13],
 3: [],
 4: [12],
 5: [13],
 6: [11, 12, 13],
 7: [11, 12],
 8: [11, 12, 13],
 9: [12, 13],
 10: []}

We can now formulate the constraints:
- This constraints impose that each customer $c \in N_C$ is served either at home ($\sum_{l \in N_L^c}w_{cl} = 0$) or at a locker station ($\sum_{l \in N_L^c}w_{cl} = 1$), hence customers are included in a tour only when they are not travelling to any locker.
$$\sum_{(i,c) \in \delta^-(c)}x_{ic} = \sum_{(c,i) \in \delta^+(c)}x_{ci} = 1 - \sum_{l \in N_L^c}w_{cl} \quad \forall c \in N_C$$

In [450]:
for c in N_C:
    # Primo membro: somma degli archi entranti nel cliente c
    incoming_arcs = gp.quicksum(x[i, c] for i in N if (i, c) in A)  # δ^-(c)
    
    # Secondo membro: somma degli archi uscenti dal cliente c
    outgoing_arcs = gp.quicksum(x[c, i] for i in N if (c, i) in A)  # δ^+(c)
    
    # Somma delle variabili w[c, l] per i locker che possono servire il cliente c
    lockers_used = gp.quicksum(w[c, l] for l in N_L_c[c])
    
    # Vincolo: cliente servito direttamente o tramite locker
    model.addConstr(incoming_arcs == outgoing_arcs, name=f"flow_conservation_c_{c}")
    model.addConstr(incoming_arcs == 1 - lockers_used, name=f"served_directly_or_locker_c_{c}")

- This constraint set the inclusion of lockers into routes, stating that a vehicle needs to visit a locker only if it is open.
$$\sum_{(i,l)\in \delta^-(l)}x_{il} = \sum_{(l,i)\in \delta^+(l)}x_{li} = y_l \quad \forall l \in N_L

In [451]:
for l in N_L:
    # Somma degli archi che entrano nel locker l
    model.addConstr(
        gp.quicksum(x[i, l] for i in N if i != l) == y[l],
        name=f"visit_locker_in_{l}"
    )

    # Somma degli archi che escono dal locker l
    model.addConstr(
        gp.quicksum(x[l, i] for i in N if i != l) == y[l],
        name=f"visit_locker_out_{l}"
    )

- This constraint ensures that at most $K$ vehicles are used
$$\sum_{(0,j)\in \delta^+(0)}x_{0j} = \sum_{(j,0)\in \delta^-(0)}x_{j0} \le |K|

In [452]:
model.addConstr(
    gp.quicksum(x[0, j] for j in N if j != 0) == gp.quicksum(x[j, 0] for j in N if j != 0),
    name="balance_vehicles_in_out"
)

model.addConstr(
    gp.quicksum(x[0, j] for j in N if j != 0) <= K,
    name="max_vehicles"
)

<gurobi.Constr *Awaiting Model Update*>

- This constraint statrs that a client can travel to a locker only if it has been opened and served by one vehicle route
$$w_{cl} \le y_l \quad \forall c \in N_C, l \in N_L^c$$

In [453]:
for c in N_C:
    for l in N_L_c[c]:  # Solo i lockers accessibili dal cliente c
        model.addConstr(
            w[c, l] <= y[l],
            name=f"client_to_locker_{c}_{l}"
        )

- This constraint determines the arrival time at two consecutive nodes, thus working as sub - tour elimination constraints. In particular, if node $j$ is visited immediately after node $i$, the time elapsed between the arrival in the two nodes is equal to the time $t_{ij}$ needed to travel between the two nodes plus the service time at node $i$.
$$\sum_{(i,j)\in\delta^+(i)}z_{ij} - \sum_{(j,i)\in\delta^-(i)}z_{ji} = \sum_{(i,j)\in\delta^+(i)}(t_{ij} + t_i)x_{ij} \quad \forall i \in N_L \cup N_C

In [454]:
for i in N_L + N_C:
    model.addConstr(
        gp.quicksum(z[i, j] for j in N if (i, j) in A) - 
        gp.quicksum(z[j, i] for j in N if (j, i) in A) ==
        gp.quicksum((distance(i, j) + t[i]) * x[i, j] for j in N if (i, j) in A),
        name=f"arrival_time_constraint_{i}"
    )

- This constraint sets the lower and upper bound of variable $z_{ij}$. If arc $(i,j)$ is traversed, then the arrival time at node $j$ must be greater than the time required to leave the depot and serve the customer $i$ ($t_{0,i} + t_{i}$) and lower than the allowed tour length ($T$) minus the time required to serve the client $j$ and return to the depot ($t_j + t_{j0}$)
$$(t_{0i} + t_{ij} + t_i)x_{ij} \le z_{ij} \le (T - t_{j0} - t_j)x_{ij} \quad \forall (i,j) \in A$$

In [455]:
T = 100  # lunghezza massima del tour

# Aggiungi vincoli per ogni arco (i, j) ∈ A
for i, j in A:
    if i != j:
        # Vincolo inferiore: (t_{0i} + t_{ij} + t_i)x_{ij} ≤ z_{ij}
        model.addConstr(
            (distance(0, i) + distance(i, j) + t[i]) * x[i, j] <= z[i, j],
            name=f"lower_bound_z_{i}_{j}"
        )
        
        # Vincolo superiore: z_{ij} ≤ (T - t_{j0} - t_j)x_{ij}
        model.addConstr(
            z[i, j] <= (T - distance(j, 0) - t[j]) * x[i, j],
            name=f"upper_bound_z_{i}_{j}"
        )

- This constraint ensures that the time needed to travel from the depot to any visited node $i$ (when $x_{0i} = 1$) is equal to $t_{0i}$
$$z_{0i} = t_{0i}x_{0i} \quad \forall i \in N_L \cup N_C$$

In [456]:
for i in N_L + N_C:
    model.addConstr(z[0, i] == distance(0, i) * x[0, i], name=f"time_depot_to_{i}")

We define a function in order to print the final route.

In [457]:
def print_route_and_objective(solution_x, solution_z, N, N_C, N_L, model, time_windows):
    # Teniamo traccia di quali nodi sono già stati visitati
    visited = set()
    
    # Inizia dal deposito (nodo 0)
    current_node = 0
    route = [current_node]
    
    while True:
        # Trova il prossimo nodo che è collegato al nodo corrente con x_{current_node, j} = 1
        next_node = None
        for j in N:
            if current_node != j and solution_x[current_node, j] > 0.5 and j not in visited:
                next_node = j
                break
        
        if next_node is None:
            # Nessun nodo successivo trovato, ritorna al deposito
            break
        
        # Aggiungi il nodo successivo alla rotta e segna come visitato
        route.append(next_node)
        visited.add(next_node)
        current_node = next_node
    
    # Aggiungi il ritorno al deposito se necessario
    if current_node != 0:
        route.append(0)
    
    # Stampa il percorso
    print("Percorso ottimale del veicolo:")
    print(" -> ".join(map(str, route)))
    
    # Stampa il valore della funzione obiettivo
    print(f"Valore della funzione obiettivo: {model.objVal}")
    
    # Stampa i tempi di arrivo confrontati con le finestre temporali
    print("\nTempi di arrivo ai clienti e confronto con le finestre temporali:")
    for c in N_C:
        a_c, b_c = time_windows[c]
        arrival_time = None
        for i in N:
            if (i, c) in solution_z and solution_x[i, c] == 1:
                arrival_time = solution_z[i, c]
                break
        if arrival_time is not None:
            print(f"Cliente {c}: tempo di arrivo = {arrival_time}, finestra temporale = [{a_c}, {b_c}]")
            if arrival_time < a_c:
                print(f" -> Arrivo anticipato rispetto alla finestra temporale!")
            elif arrival_time > b_c:
                print(f" -> Arrivo in ritardo rispetto alla finestra temporale!")
        else:
            print(f"Cliente {c}: nessun arrivo registrato")

    # Stampa quali clienti vanno a un locker
    print("\nClienti che vanno a un locker:")
    for c in N_C:
        for l in N_L_c[c]:
            if w[c, l].X > 0.5:  # Se il cliente c va al locker l
                print(f"Cliente {c} va al locker {l}")


In [458]:
for c in N_C:
    a_c, b_c = time_windows[c]
    
    # Vincolo: il veicolo non può arrivare prima di a_c
    model.addConstr(
        gp.quicksum(z[i, c] for i in N if (i, c) in A) >= a_c, 
        name=f"time_window_lower_bound_{c}"
    )
    
    # Vincolo: il veicolo non può arrivare dopo b_c
    model.addConstr(
        gp.quicksum(z[i, c] for i in N if (i, c) in A) <= b_c,
        name=f"time_window_upper_bound_{c}"
    )


In [459]:
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i3-1000NG4 CPU @ 1.10GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 451 rows, 439 columns and 2073 nonzeros
Model fingerprint: 0xba9b666a
Variable types: 210 continuous, 229 integer (229 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 4e+01]
Presolve removed 111 rows and 128 columns
Presolve time: 0.01s
Presolved: 340 rows, 311 columns, 1555 nonzeros
Variable types: 138 continuous, 173 integer (173 binary)

Root relaxation: objective 2.575560e+01, 158 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   25.75560    0   26          -   25.75560      -    

In [460]:

# Dopo l'ottimizzazione
if model.status == GRB.OPTIMAL:
    # Estrai i valori ottimali delle variabili x_{ij} e z_{ij}
    solution_x = model.getAttr('x', x)
    solution_z = model.getAttr('x', z)
    
    # Stampa il tragitto del veicolo, i tempi di arrivo e il confronto con le time windows
    print_route_and_objective(solution_x, solution_z, N, N_C, N_L, model, time_windows)


Percorso ottimale del veicolo:
0 -> 1 -> 10 -> 4 -> 0 -> 5 -> 13 -> 8 -> 3 -> 6 -> 7 -> 0
Valore della funzione obiettivo: 57.0

Tempi di arrivo ai clienti e confronto con le finestre temporali:
Cliente 1: tempo di arrivo = 13.0, finestra temporale = [0, 40]
Cliente 2: nessun arrivo registrato
Cliente 3: tempo di arrivo = 26.0, finestra temporale = [0, 40]
Cliente 4: tempo di arrivo = 34.0, finestra temporale = [0, 40]
Cliente 5: tempo di arrivo = 4.0, finestra temporale = [0, 40]
Cliente 6: tempo di arrivo = 29.0, finestra temporale = [0, 40]
Cliente 7: tempo di arrivo = 34.0, finestra temporale = [0, 40]
Cliente 8: tempo di arrivo = 19.0, finestra temporale = [0, 40]
Cliente 9: nessun arrivo registrato
Cliente 10: tempo di arrivo = 23.0, finestra temporale = [0, 40]

Clienti che vanno a un locker:
Cliente 2 va al locker 13
Cliente 9 va al locker 13


In [461]:
distance(0,13)

5

In [462]:
distance(13,10)

15

In [463]:
t[13]

2